### 대조약정보 검색하기
* 대조약 정보(메인페이지 → 의약품등 정보 → 제네릭의약품 → 대조약정보) https://nedrug.mfds.go.kr/pbp/CCBBR01

In [1]:
import pandas as pd

In [2]:
url = "https://nedrug.mfds.go.kr/pbp/CCBBR01"

### 검색 값 설정하기
* 웹사이트에서 검색한 URL 안에 우리의 검색어가 들어가 있습니다.
* 이렇게 검색어를 URL에 붙여서 넘기는 문구를 쿼리스트링이라고 부릅니다.
* 이 문자열을 검색어에 맞게 변경해 줍니다.

In [3]:
page_no = 1
base_url = "https://nedrug.mfds.go.kr/pbp/CCBBR01/getList?"
params = f"totalPages=22&page={page_no}&limit=100&sort=&sortOrder="
params = f"{params}&searchYn=false&startDateStart=&startDateEnd="
params = f"{params}&searchGbn=itemName&search="
params

'totalPages=22&page=1&limit=100&sort=&sortOrder=&searchYn=false&startDateStart=&startDateEnd=&searchGbn=itemName&search='

In [4]:
# base_url 과 params를 합쳐서 url 만들기
# url = base_url + params
url = f"{base_url}{params}"
url

'https://nedrug.mfds.go.kr/pbp/CCBBR01/getList?totalPages=22&page=1&limit=100&sort=&sortOrder=&searchYn=false&startDateStart=&startDateEnd=&searchGbn=itemName&search='

### read_html 로 웹사이트의 내용 읽어오기
* html 소스코드 내부에 < table > 로 된 태그가 있을 때만 가능합니다.

In [5]:
# read_html 로 데이터 가져오기
temp = pd.read_html(url, encoding="utf-8")
len(temp)

1

### 페이지번호, 검색어를 넘기면 데이터를 가져오는 함수 만들기

In [6]:
# try, except 를 사용해서 오류가 났을 때 오류대신 메시지만 출력하도록 합니다.
def get_page_content(page_no):
    try:
        base_url = "https://nedrug.mfds.go.kr/pbp/CCBBR01/getList?"
        params = f"totalPages=22&page={page_no}&limit=100&sort=&sortOrder="
        params = f"{params}&searchYn=false&startDateStart=&startDateEnd="
        params = f"{params}&searchGbn=itemName&search="
        url = f"{base_url}{params}"
        table = pd.read_html(url, encoding="utf-8")
        return table[0]
    except Exception as e:
        return e

In [7]:
# 함수 확인하기
get_page_content(page_no)

,순번,성분명,제품명,업체명,구분,제형,공고일
0,1,로사르탄칼륨/암로디핀캄실산염,아모잘탄정5/50밀리그램,한미약품(주),생동대조약,필름코팅정,2019-09-30
1,2,로사르탄칼륨/암로디핀캄실산염,아모잘탄정5/100밀리그램,한미약품(주),생동대조약,필름코팅정,2019-09-30
2,3,부프레노르핀,노스판패취10㎍/h(부프레노르핀),한국먼디파마 유한회사,NaN,서방성경피흡수제,2019-06-27
3,4,레보플록사신수화물,크라비트점안액1.5%(레보플록사신수화물),한국산텐제약(주),이동대조약,점안용액제,2019-06-27
4,5,결합형에스트로겐4.29%/바제독시펜아세테이트,듀아비브정0.45/20밀리그램,한국화이자제약(주),생동대조약,서방성필름코팅정,2019-06-27
...,...,...,...,...,...,...,...
95,96,로수바스타틴칼슘/에제티미브,로수젯정10/5밀리그램,한미약품(주),NaN,필름코팅정,2018-03-30
96,97,로수바스타틴칼슘/에제티미브,로수젯정10/10밀리그램,한미약품(주),NaN,필름코팅정,2018-03-30
97,98,로수바스타틴칼슘/에제티미브,로수젯정10/20밀리그램,한미약품(주),NaN,필름코팅정,2018-03-30
98,99,로수바스타틴칼슘/텔미사르탄,듀오웰정80/20밀리그램,(주)유한양행,NaN,필름코팅정,2018-03-30


### 반복문을 통해 전체 페이지를 수집

* tqdm은?
    * 오래 걸리는 작업의 진행 상태를 표시합니다.
    * https://tqdm.github.io/

In [8]:
# tqdm 불러오기
from tqdm import trange
# 페이지마다의 목록을 담아줄 리스트 초기화
page_list = []

# 반복문을 통해 여러 페이지의 목록 수집하기
# for page_no in trange(1, 23):
#     temp = get_page_content(page_no)
#     page_list.append(temp)

In [9]:
# 정상동작 확인
page_no = 1
temp_1 = get_page_content(page_no)
temp_1.shape

(100, 7)

In [10]:
# 없는 페이지 접근 확인
page_no = 23
temp_2 = get_page_content(page_no)
print(temp_2.shape)
temp_2.iloc[0, 0]

(1, 7)


'조회 결과가 없습니다.'

In [11]:
import time
page_no = 1
page_list = []
while True:
    temp = get_page_content(page_no)
    if temp.shape[0] > 1:
        page_list.append(temp)
        page_no = page_no + 1
        time.sleep(0.1)
        if page_no % 10 == 0: # 10페이지 단위로만 출력하기
            print(page_no)
    else:
        break

10
20


### 수집한 데이터 하나의 데이터프레임으로 합치기


<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [12]:
# concat으로 데이터프레임 하나로 만들기
df = pd.concat(page_list)
df.shape

(2121, 7)

### 파일로 저장하기

In [13]:
import datetime
today = datetime.datetime.today().strftime("%Y-%m-%d")
today
# today = "2021-06-04"

'2021-06-04'

In [14]:
file_name = f"대조약정보-{today}.csv"
file_name

'대조약정보-2021-06-04.csv'

In [15]:
# to_csv로 저장하기
df.to_csv(file_name, index=False, encoding="cp949")

In [16]:
# read_csv로 읽어오기
pd.read_csv(file_name, encoding="cp949")

,순번,성분명,제품명,업체명,구분,제형,공고일
0,1,로사르탄칼륨/암로디핀캄실산염,아모잘탄정5/50밀리그램,한미약품(주),생동대조약,필름코팅정,2019-09-30
1,2,로사르탄칼륨/암로디핀캄실산염,아모잘탄정5/100밀리그램,한미약품(주),생동대조약,필름코팅정,2019-09-30
2,3,부프레노르핀,노스판패취10㎍/h(부프레노르핀),한국먼디파마 유한회사,NaN,서방성경피흡수제,2019-06-27
3,4,레보플록사신수화물,크라비트점안액1.5%(레보플록사신수화물),한국산텐제약(주),이동대조약,점안용액제,2019-06-27
4,5,결합형에스트로겐4.29%/바제독시펜아세테이트,듀아비브정0.45/20밀리그램,한국화이자제약(주),생동대조약,서방성필름코팅정,2019-06-27
...,...,...,...,...,...,...,...
2116,2117,딜티아젬염산염,유영딜티아젬정30mg(딜티아젬염산염)(수출용),(주)유영제약,NaN,나정,2002-11-22
2117,2118,라록시펜염산염,에비스타정60밀리그람(라록시펜염산염),알보젠코리아(주),생동대조약,필름코팅정,2002-11-22
2118,2119,라미부딘,제픽스정100밀리그램(라미부딘),(주)글락소스미스클라인,생동대조약,필름코팅정,2002-11-22
2119,2120,테오필린서방형입자,보령오스틴서방캅셀(테오필린),보령제약(주),생동대조약,"서방성캡슐제, 펠렛",2002-11-22
